<a href="https://colab.research.google.com/github/miahong/Colonoscopy-AI/blob/master/colonoscopy_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Data exploration and data proprocessing
import shutil
import time
import copy
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models

In [0]:
# get both ungrouped AO and Non-AO image paths and check the total number of images
AO_folder_path = "/home/hongzhuoqiao/Colonoscopy-AI/data/model_data/AO/*"
NonAO_folder_path = "/home/hongzhuoqiao/Colonoscopy-AI/data/model_data/Non-AO/*"
AO_files = glob.glob(AO_folder_path)
NonAO_files = glob.glob(NonAO_folder_path)

# AO image original path
AO_img_path_list =[]
for file in AO_files:
    filename = os.path.basename(file)
    AO_img_path_list.append(file)

# Non AO image original path
NonAO_img_path_list =[]
for file in NonAO_files:
    filename = os.path.basename(file)
    NonAO_img_path_list.append(file)

# Read and Store Each Images and shapes
# AO image and shape
AO_img_list =[]
AO_img_shape_list =[]
for path in AO_img_path_list[0:1]:
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    AO_img_list.append(image)
    AO_img_shape_list.append(image.shape)
 
# Non-AO image and shape
NonAO_img_list =[]
NonAO_img_shape_list =[]
for path in NonAO_img_path_list[0:1]:
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    NonAO_img_list.append(image)
    NonAO_img_shape_list.append(image.shape)

# check unique shape size
# AO unique shape
AO_unique_shape_list =[]
for s in AO_img_shape_list:
    if s not in AO_unique_shape_list:
        AO_unique_shape_list.append(s)

NonAO_unique_shape_list =[]
for s in NonAO_img_shape_list:
    if s not in NonAO_unique_shape_list:
        NonAO_unique_shape_list.append(s)

AO_count = len(AO_img_path_list)
NonAO_count = len(NonAO_img_path_list)
total_count = AO_count + NonAO_count
print ("AO images have shape: {}".format(AO_unique_shape_list))
print ("Non AO images have shape: {}".format(NonAO_unique_shape_list))
print ("Overall, {} AO images, {} Non AO images, totally {} images.".format(AO_count, NonAO_count, total_count))
print ("potential train count: {} ".format(int(total_count/10*8)+1))
# plt.imshow(NonAO_img_list[0])
# plt.show()

AO images have shape: [(480, 736, 3)]
Non AO images have shape: [(480, 720, 3)]
Overall, 6559 AO images, 6663 Non AO images, totally 13222 images.
potential train count: 10578 


In [0]:
# Data_loader for modeling

BATCH_SIZE = 32
NUM_WORKDER =4
# Build transforms
data_transform = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Build the datasets
model_dataset = datasets.ImageFolder(root='data/model_data', transform=data_transform)
len(model_dataset)

# Split the dataset into train valid and test dataset
train_count = int(total_count/10*8) +1 
valid_count = int((total_count - train_count)/2)
test_count = int((total_count - train_count)/2)
print('train, valid, test split count:{} {} {} '.format(train_count, valid_count, test_count))
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(model_dataset, [train_count, valid_count, test_count])

# Build Dataloader for three datasets, same dataloader for three datasets
train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKDER)
valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKDER)
test_dataset_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKDER)
print ("Dataloader have been built.")

train, valid, test split count:10578 1322 1322 
Dataloader have been built.


In [0]:
# Build train model with pretrain model
LEARNING_RATE = 0.001
MOMENTUM = 0.9
STEP_SIZE = 7
GAMMA = 0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = models.resnet18(pretrained=True)
num_features = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_features, 2)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=LEARNING_RATE, momentum = MOMENTUM)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = STEP_SIZE, gamma=GAMMA)

In [0]:
# training model function
# TODO: change the function to adjust the current dataloader 
# reference to https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#training-the-model

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        #TODO: change here
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
# N_EPOCHS = 25
# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=N_EPOCHS)